<a href="https://colab.research.google.com/github/Gan4x4/CV-HSE2019/blob/master/video/Action_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Download labels for Keenetic - 400
!wget https://gist.githubusercontent.com/willprice/f19da185c9c5f32847134b87c1960769/raw/9dc94028ecced572f302225c49fcdee2f3d748d8/kinetics_400_labels.csv -O kinetics_400_labels.csv

# Download youtube-dl
!sudo wget https://yt-dl.org/downloads/latest/youtube-dl

# Make youtube-dl executable
!sudo chmod a+rx youtube-dl

# Download "Scent of a woman" trailer
! ./youtube-dl -f 134 https://www.youtube.com/watch?v=losLAzU9YCk -o video.mp4

--2020-03-16 13:43:35--  https://gist.githubusercontent.com/willprice/f19da185c9c5f32847134b87c1960769/raw/9dc94028ecced572f302225c49fcdee2f3d748d8/kinetics_400_labels.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7454 (7.3K) [text/plain]
Saving to: ‘kinetics_400_labels.csv’

kinetics_400_labels 100%[===================>]   7.28K  --.-KB/s    in 0s      

2020-03-16 13:43:35 (47.6 MB/s) - ‘kinetics_400_labels.csv’ saved [7454/7454]

--2020-03-16 13:43:37--  https://yt-dl.org/downloads/latest/youtube-dl
Resolving yt-dl.org (yt-dl.org)... 95.143.172.170, 2001:1a50:11:0:5f:8f:acaa:177
Connecting to yt-dl.org (yt-dl.org)|95.143.172.170|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://yt-dl.org/downloads/2020.03.08/youtu

In [0]:
import csv
# Read labels from Keenetic - 400 dataset
# becouse PyTorch models trained on it

def read_labels():
    labels = {}
    with open('kinetics_400_labels.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        i = 0
        for col in reader:
            if i == 0:
                print(';'.join(col))
            else:
                labels[int(col[0])] = col[1]
            i += 1
    return labels


In [0]:
# Not worked :(
import torchvision
fragment = torchvision.io.read_video("video.mp4", 0, 64, pts_unit='pts')

ImportError: ignored

In [0]:
import cv2
import torch
from torch.utils.data import DataLoader
from torch.utils.data import IterableDataset
from torchvision import models, datasets, transforms
from PIL import Image
import csv

class VideoDataset(IterableDataset):
    def __init__(self, filename, transform):
        super().__init__()
        self.transform = transform
        self.stream = cv2.VideoCapture(filename)

    def read_next_frame(self):
        while True:
            samples = torch.zeros(0,3,112,112)
            for i in range(16):
                ret, frame = self.stream.read()
                if not ret:
                  self.stream.release()
                  return None
                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                im_pil = Image.fromarray(img)
                tmp = self.transform(im_pil).unsqueeze(0)
                samples = torch.cat((samples,tmp))
            samples = samples.permute((1,0,2,3))
            yield torch.tensor(samples)

    def __iter__(self):
        # worker_info = torch.utils.data.get_worker_info()
        # assert worker_info is None:  # single-process data loading, return the full iterator
        return self.read_next_frame()


labels = read_labels()

resnet18_3d = models.video.r3d_18(pretrained=True)
resnet18_3d.cuda()
resnet18_3d.eval()

# https://pytorch.org/docs/stable/torchvision/models.html#video-classification
# ... models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB videos of shape 
#(3 x T x H x W),
# where H and W are expected to be 112, and T is a number of video frames in a clip.
# The images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.43216, 0.394666, 0.37645] 
# and std = [0.22803, 0.22145, 0.216989].

transform = transforms.Compose([
                                transforms.Resize((112,112)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.43216, 0.394666, 0.37645], [0.22803, 0.22145, 0.216989])])

vd = VideoDataset('video.mp4', transform)
dataloader = DataLoader(vd)
subtitles = {}
packet_size = 16
i = 0
conf_threshold = 0.45
for batch in dataloader:
  if batch is None:
    break
  with torch.no_grad():
    result = resnet18_3d(batch.cuda())
    sm = torch.nn.Softmax(dim = 1) # 0 - is batch size
    result = sm(result)
    score, id = result.max(1)
    print(labels[int(id.detach().item())], score.item())
    if score > conf_threshold: # Use only predictions with confidence above threshold
      subtitles[i*packet_size] = "{} {:.2f}".format(labels[int(id.item())],score.detach().item())
    else:
      subtitles[i*packet_size] = ""
    i += 1
   

id;name


Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /root/.cache/torch/checkpoints/r3d_18-b3b3357e.pth


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


hammer throw 0.06372684240341187
tai chi 0.38046014308929443
tai chi 0.3615225851535797
tai chi 0.3678002953529358
tai chi 0.40854644775390625
tai chi 0.3730301856994629
tai chi 0.4090980887413025
tai chi 0.38700437545776367
tai chi 0.37682491540908813
tai chi 0.19246824085712433
dancing ballet 0.08317755162715912
golf driving 0.3107621669769287
bowling 0.28593742847442627
bowling 0.3495776355266571
bowling 0.19746829569339752
diving cliff 0.07870832830667496
parkour 0.31560102105140686
driving tractor 0.4786298871040344
disc golfing 0.12461186945438385
parkour 0.50790935754776
playing saxophone 0.22070284187793732
waiting in line 0.3357901871204376
busking 0.24348442256450653
giving or receiving award 0.6048723459243774
giving or receiving award 0.8831714391708374
giving or receiving award 0.2689860761165619
slapping 0.1498323380947113
reading newspaper 0.21372276544570923
texting 0.49530866742134094
texting 0.34102845191955566
busking 0.6346886157989502
busking 0.4868960976600647
rid

In [0]:
print(subtitles)

{0: '', 16: '', 32: '', 48: '', 64: '', 80: '', 96: '', 112: '', 128: '', 144: '', 160: '', 176: '', 192: '', 208: '', 224: '', 240: '', 256: '', 272: 'driving tractor 0.48', 288: '', 304: 'parkour 0.51', 320: '', 336: '', 352: '', 368: 'giving or receiving award 0.60', 384: 'giving or receiving award 0.88', 400: '', 416: '', 432: '', 448: 'texting 0.50', 464: '', 480: 'busking 0.63', 496: 'busking 0.49', 512: 'riding unicycle 0.53', 528: 'pushing wheelchair 0.70', 544: '', 560: '', 576: '', 592: '', 608: '', 624: 'playing chess 0.69', 640: 'playing chess 0.46', 656: 'shaking hands 0.48', 672: 'slapping 0.47', 688: '', 704: 'cleaning windows 0.47', 720: '', 736: 'digging 0.51', 752: '', 768: 'chopping wood 0.77', 784: 'chopping wood 0.76', 800: '', 816: 'fixing hair 0.71', 832: 'fixing hair 0.78', 848: '', 864: '', 880: '', 896: '', 912: '', 928: '', 944: '', 960: '', 976: '', 992: '', 1008: '', 1024: 'smoking 0.51', 1040: '', 1056: '', 1072: '', 1088: 'smoking 0.49', 1104: 'reading bo

In [0]:
# Write predictions to video as subtitles
stream = cv2.VideoCapture("video.mp4")
fps = stream.get(cv2.CAP_PROP_FPS)
width  = int(stream.get(cv2.CAP_PROP_FRAME_WIDTH))   
height = int(stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4',fourcc,fps, (width,height))
frame_num = 0
current_label =""
while stream.isOpened():
   ret, frame = stream.read() 
   current_label = subtitles.get(frame_num,current_label)
   if ret:
     frame = cv2.putText(frame, current_label, (20,300), cv2.FONT_HERSHEY_SIMPLEX , 1, (255,0,0), 3, cv2.LINE_AA) 
     out.write(frame)
   else:
     break
   frame_num += 1
   if not frame_num % 250:
     print ("Writed frame",frame_num)
  
stream.release()
out.release()